## Алгоритм Крикпатрика

### Идея

   Алгоритм Киркпатрика работает с <b>PSLG</b>, представляющими собой триангуляцию (за $O(n\log{n})$ заметающей прямой можно триангулировать любой PSLG, так что с этим проблем нет).
   <br>
   Идея: давайте построим последовательность триангуляций, где первая триангуляция совсем простая, а каждая следующая – посложнее, но ненамного. Каждый треугольник в простой триангуляции знает, какие треугольники в более сложной он пересекает. Локализовавшись в простой триангуляции, мы сможем спускаясь все ниже и ниже уровнем дойти до искомой триангуляции и локализоваться в ней.
   Вопрос только в том, как строить эти уровни так, чтобы их было не слишком много и чтобы переходить от уровня к уровню было не слишком сложно.
   
   Напомним, что триангуляция на множестве вершин $V$ есть планарный граф с не более чем $3 |V| - 6$ ребрами (формула Эйлера). 
   <br>
   Для удобства реализации соединим все точки границы триангуляции с некоторой бесконечно удаленной точкой. Это можно сделать, например, переместив весь <b>PSLG</b> на плоскость $z = 1$, тогда бесконечно удаленная точка будет иметь координаты $(0,0,0)$. После этого преобразования все триангуляции будут обладать тремя границами и ровно $3 |V| - 6$ ребрами.
   
### Структура данных
   
   Итак, имеется N-вершинная триангуляция $G$, и пусть строится последовательность триангуляций $S_1, S_2, \dots, S_{h(N)}$, где $S_1 = G$, а $S_i$ получается из $S_{i - 1}$ по следующим правилам:
* <b>Шаг 1.</b> Удалим некоторое количество независимых(попарно несмежных друг с другом) вершин(кроме бесконечно удаленной) и инцидентные им ребра (от выбора этого множества напрямую зависит оптимальность алгоритма).
* <b>Шаг 2.</b> Построить триангуляцию получившихся в результате шага $1$ многоугольников.

Таким образом $S_{h(N)}$ состоит из одного треугольника. Далее, будем обозначать все треугольники как $R$, а также будем говорить, что треугольник $R_{ij}$ принадлежит триангуляции $S_i$, если 
он был создан на шаге (2) при построении этой триангуляции.

Теперь построим структуру данных $T$ для поиска. Эта структура представляет собой направленный ацикличный граф, вершинами которого будут наши треугольники. Определим эту структуру следующим образом: из треугольника $R_k$ будет вести ребро в треугольник $R_j$, если при построении $S_i$ из $S_{i-1}$ мы имеем
* $R_j$ удалятся из $S_{i - 1}$ на шаге (1)
* $R_k$ создается в $S_{i}$ на шаге (2)
* $R_j \cap R_k \ne  \varnothing $

Очевидно, что треугольники из $S_1$ (и только они) не имеют исходящих ребер.

### Выбор множества удаляемых вершин

Как уже упоминалось, от выбора множества вершин триангуляции, которые будут удалены при построении $S_i$ по $S_{i-1}$ существенно зависит эффективность метода. Предположим, что можно выбрать это множество так, чтобы выполнялись следующие ''свойства'' ($N_i$ обозначает число вершин в $S_i$):

<b>Свойство 1:</b> $N_i = a_i N_{i-1}$, где $a_i \le a < 1$ для $i = 2,\dots , h(N)$.

<b>Свойство 2:</b> Каждый треугольник $R_i \in S_i$ пересекается не более чем с $H$ треугольниками из $S_{i-1}$ и наоборот.
Первое свойство немедленно влечет за собой следствие, что $h(N) \le \left \lceil \log_{1/a}N \right \rceil = O(log N)$, поскольку при переходе от $S_{i-1}$ к $S_i$ удаляется по меньшей мере фиксированная доля вершин.


Также из этих свойств следует, что <b>память</b>, занимаемая $T$, равна $O(N)$.
#### Оценка занимаемой памяти
Действительно, заметим, что эта память используется для хранения узлов и указателей на их потомков. Из формулы Эйлера следует, что $S_i$ содержит $F_i < 2N_i$ треугольников. Число узлов в $T$, представляющих треугольники из $S_i$, не превосходит $F_i$ (только те треугольники, которые действительно принадлежат $S_i$, появляются на соответствующем «ярусе» $T$). Отсюда следует, что общее число узлов в $T$ меньше, чем
<br>
$2(N_1 + N_2 + \dots + N_{h(N)}) \le 2N_1(1 + a + a^2 + \dots + a^{h(N) - 1}) < \frac{2N}{1 - a}$.

Что касается памяти, используемой под указатели, то по <b>свойству 2</b> каждый узел имеет не более $H$ указателей, поэтому не более $\frac{2NH}{1-a}$ указателей появится в $T$. Это доказывает последнее утверждение.

### Критерий выбора множества удаляемых вершин
Покажем теперь, что критерий выбора множества удаляемых вершин, удовлетворяющий вышеописанным свойствам, существует.
<br>
Если на <b>шаге 1</b> построения последовательности триангуляции удалять несмежные вершины со степенью меньше некоторого целого числа $K$(будет указано позже), то свойства, описанные выше, будут выполнены.

<b>Доказательство критерия</b>

<b>1.</b> Для проверки первого свойства воспользуемся некоторыми особенностями плоских графов. Из формулы Эйлера для плоских графов, в частном случае триангуляции, ограниченной тремя ребрами, следует, что число вершин $N$ и число ребер $E$ связаны соотношением 
$E = 3N - 6$.

Пока в триангнуляции есть внутренние вершины (в противном случае задача тривиальна), степень каждой из трех граничных вершин не меньше трех. Поскольку существует $3N - 6$ ребер, а каждое ребро инцидентно двум вершинам, то сумма степеней всех вершин меньше $6N$. Отсюда сразу следует, что не менее $ \dfrac{N}{2}$ вершин имеет степень меньше $12$. Тогда пусть $K = 12$. 

Пусть $M\ -$ число выбранных вершин. Поскольку каждой из них инцидентно не более $K-1 = 11$ ребер, а три граничные вершины не выбираются, то мы имеем 
$M \geqslant \left \lfloor \dfrac{1}{12}(\dfrac{N}{2} - 3) \right \rfloor $.

Следовательно, $a \cong 1 - \dfrac{1}{24} < 0,959 < 1$, что доказывает справедливость <b>свойства 1</b>.
<br>
<b>2.</b> Выполнение второго свойства обеспечивается тривиально. Поскольку удаление вершины со степенью меньше $K$ приводит к образованию многоугольника с числом ребер менее $K$, то каждый из удаленных треугольников пересекает не более $K - 2 = H$ новых треугольников.

### Сложность алгортма
Алгоритм Киркпатрика требует $O(\log{n})$ времени на запрос, $O(n)$ памяти и $O(n \log{n})$ времени на для построения поисковой структуры.

<b>Доказательство</b>

Оценки на память и время запроса мы доказали выше. Чтобы доказать оценку на построение структуры, нужно доказать, что построение очередного слоя работает за $O(n)$: так как всего слоев $O(\log{n})$, это повлечет за собой искомую оценку.
<br>
Выбрать все вершины для удаления мы можем, очевидно, за $O(n)$. После удаления вершин останется $V$ многоугольников, которые нужно триангулировать. Однако заметим, что все эти треугольники - <b>звездные</b>, поэтому каждый из них можно триангулировать за $O(m)$, где $m$ - число вершин в многоугольнике. Суммарное число вершин в многоугольниках - $O(n)$, поэтому они все будут триангулированы за $O(n)$.